# To find the ground state of the hydrogen molecule, $H_2$

In [1]:
import numpy as np

In [2]:
symbols = ['H', 'H']
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

In [3]:
import pennylane as qml

In [4]:
Hamiltonian, Nqubits = qml.qchem.molecular_hamiltonian(symbols, coordinates) 

In [5]:
print("Number of qubits = ", Nqubits)
print("The Hamiltonian is ", Hamiltonian)

Number of qubits =  4
The Hamiltonian is    (-0.24274501727498227) [Z2]
+ (-0.24274501727498227) [Z3]
+ (-0.042072543031530185) [I0]
+ (0.17771358191549919) [Z0]
+ (0.1777135819154993) [Z1]
+ (0.12293330460167415) [Z0 Z2]
+ (0.12293330460167415) [Z1 Z3]
+ (0.16768338881432718) [Z0 Z3]
+ (0.16768338881432718) [Z1 Z2]
+ (0.1705975924056083) [Z0 Z1]
+ (0.17627661476093914) [Z2 Z3]
+ (-0.04475008421265302) [Y0 Y1 X2 X3]
+ (-0.04475008421265302) [X0 X1 Y2 Y3]
+ (0.04475008421265302) [Y0 X1 X2 Y3]
+ (0.04475008421265302) [X0 Y1 Y2 X3]


In [6]:
dev = qml.device("default.qubit", wires=Nqubits)

In [7]:
electrons = 2
hf = qml.qchem.hf_state(electrons, Nqubits)
print(hf)

[1 1 0 0]


In [8]:
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0,1,2,3])

In [9]:
@qml.qnode(dev)
def cost_fn(param):
    circuit(param, wires=range(Nqubits))
    return qml.expval(Hamiltonian)

In [10]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [11]:
theta = 0.0

In [12]:
energy = [cost_fn(theta)]
angle = [theta]
max_iterations = 100
conv_tol = 1e-06

In [13]:
for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)
    energy.append(cost_fn(theta))
    angle.append(theta)
    
    conv = np.abs(energy[-1] - prev_energy)
    
    if n%2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")
    
    if conv <= conv_tol:
        break
        
print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")    

Step = 0,  Energy = -1.12799983 Ha
Step = 2,  Energy = -1.13466246 Ha
Step = 4,  Energy = -1.13590595 Ha
Step = 6,  Energy = -1.13613667 Ha
Step = 8,  Energy = -1.13617944 Ha
Step = 10,  Energy = -1.13618736 Ha
Step = 12,  Energy = -1.13618883 Ha

Final value of the ground-state energy = -1.13618883 Ha

Optimal value of the circuit parameter = 0.2089
